# Mount drive and load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Change directories 

## Load data file

In [ ]:
fname = '/content/drive/MyDrive/UCLA/teaching/114/S2021/photon_counting/alex20170611_002_12d_tit2_D190uW_A80uW.hdf5'        

In [ ]:
from pathlib import Path

In [ ]:
fname = Path(fname)

In [ ]:
assert fname.is_file(), 'File not found.'

In [ ]:
mlabel_alex = '_'.join(fname.stem.replace('pax-', '').replace('alex-', '').split('_')[:3])
mlabel_alex

'alex20170611_002_12d'

# Imports

In [ ]:
import os
import numpy as np
from IPython.display import display, HTML
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
!pip install lmfit
import lmfit; lmfit.__version__

'1.0.1'

In [ ]:
!pip install phconvert
import phconvert; phconvert.__version__

'0.8.2'

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
mpl.rcParams['font.sans-serif'].insert(0, 'Arial')

In [ ]:
!git clone https://github.com/mayeshh/FRETBursts.git

Cloning into 'FRETBursts'...
remote: Enumerating objects: 9512, done.
remote: Total 9512 (delta 0), reused 0 (delta 0), pack-reused 9512
Receiving objects: 100% (9512/9512), 5.09 MiB | 18.47 MiB/s, done.
Resolving deltas: 100% (6945/6945), done.


In [ ]:
!pip install fretbursts
from fretbursts import *

/usr/local/lib/python3.6/dist-packages/fretbursts/__init__.py:63: UserWarning:  - Cannot import QT, custom GUI widgets disabled.
  warnings.warn((' - Cannot import QT, custom GUI widgets disabled.'))


ImportError: ignored

In [ ]:
sns = init_notebook(apionly=True)

In [ ]:
# load utility function (not import) giving them access to globals ("-i" flag)
# Defines: savefig, save_name, info_html, cal_phrate, make_df_spots, make_df_bursts
#          cal_phrate_alex, make_df_bursts_alex
%run -i utils.py  

In [ ]:
save_figures = True
savefigdir = 'figures'
highres = True

# Load the selected file

We can finally load the data and store it in a variable called `d`:

In [ ]:
d = loader.photon_hdf5(fname)

In [ ]:
info_html(d)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
bpl.plot_alternation_hist_usalex(d, ax=ax, bins=np.arange(0, 4097, 16))

If the previous alternation histogram looks correct, 
the corresponding definitions of the excitation periods can be applied to the data using the following command:

In [ ]:
loader.alex_apply_period(d)

In [ ]:
num_time_points = 5
window = 5

# Timepoints equally distributed along the measurement
time_points = np.round(np.linspace(d.time_min+1, d.time_max-window-1, num=num_time_points))

for i in time_points:
    dplot(d, timetrace, tmin=i, tmax=i+window, figsize=(32, 3.5));
    plt.ylim(-100, 100)
    plt.xlim(i, i+window)

In [ ]:
d.calc_bg(bg.exp_fit, time_s=10, tail_min_us='auto', F_bg=1)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4))
dplot(d, timetrace_bg, ax=ax)

# Burst analysis

In [ ]:
d.burst_search(min_rate_cps=50e3)
d = d.select_bursts(select_bursts.time, time_s1=200)
d.num_bursts

In [ ]:
dplot(d, hist_size)
plt.xlim(None, 500)

In [ ]:
dplot(d, hist_width, bins=(0, 10, 0.05))
plt.xlim(0, 10)

In [ ]:
d.calc_max_rate(m=10, ph_sel=Ph_sel(Dex='Dem'), compact=True)

In [ ]:
recompute = True
streams = ('DexDem', 'DexAem', 'AexAem')
colors = (bpl.green, bpl.red, bpl.purple)
phrates = {}
for stream in streams:
    print(' - Computing peak photon rates for %6s stream.' % str(Ph_sel.from_str(stream)))
    cal_phrate_alex(d, stream=Ph_sel.from_str(stream), phrates=phrates, recompute=recompute)

In [ ]:
phrates[stream]

In [ ]:
phr_dd = phrates['DexDemB']
phr_dd[0]

In [ ]:
float(phr_dd.loc[phr_dd[0]>100e3].mean() - 100e3)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3.5))
bins = (0, 1000, 25)
for stream, color in zip(streams, colors):
    hist_burst_phrate(phrates[stream+'B'], bins=bins, plot_style=dict(color=color, ms=3, label=stream))
plt.xlim(0, 1100)
plt.ylim(1e-6)
plt.title(d.name)
plt.legend(['$D_{ex}D_{em}$', '$D_{ex}A_{em}$', '$A_{ex}A_{em}$'])
plt.grid()
plt.title('')
savefig('usALEX_peak_phrate', label=mlabel_alex)

In [ ]:
d.burst_sizes_ich()

In [ ]:
ds = d.select_bursts(select_bursts.size, th1=40, gamma=1)

In [ ]:
alex_jointplot(ds, vmax_fret=False, vmin=1)

In [ ]:
ds1 = d.select_bursts(select_bursts.naa, th1=20)
dss = ds1.select_bursts(select_bursts.size, th1=40)

In [ ]:
alex_jointplot(dss, vmax_fret=False, vmin=1)

In [ ]:
d2 = bext.burst_search_and_gate(d, min_rate_cps=30e3)

In [ ]:
ds2 = d2.select_bursts(select_bursts.size, th1=30)

In [ ]:
alex_jointplot(ds2, vmax_fret=False, vmin=1)

In [ ]:
vals, counts = np.unique(ds.burst_data_ich(0)['bg_period'], return_counts=True)

In [ ]:
counts

In [ ]:
plt.plot(vals, counts)
plt.ylim(0)